<a href="https://colab.research.google.com/github/utkarshsharma1/ECG_Signals/blob/master/2D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install split-folders tqdm
import split_folders

In [ ]:
split_folders.ratio('/content/drive/My Drive/Colab Notebooks/images_train', output="/content/division", seed=1337, ratio=(.70, .30)) # default values

Copying files: 134 files [01:23,  1.86 files/s]

KeyboardInterrupt: ignored

In [ ]:
split_folders.ratio('/content/drive/My Drive/Colab Notebooks/images_test', output="/content/division", seed=1337, ratio=(0,0,1))

In [ ]:
import os
#os.mkdir('../division')
path = os.listdir('/content/division')
path

In [ ]:
import os
PATH = os.path.join(os.path.dirname('/content/division'))

train_dir = os.path.join(PATH, 'division/train')
validation_dir = os.path.join(PATH, 'division/val')
test_dir = os.path.dirname('division/test')

train_apc_dir = os.path.join(train_dir, 'APC')  
train_lbb_dir = os.path.join(train_dir, 'LBB')  
train_nor_dir = os.path.join(train_dir, 'NOR')    
train_pvc_dir = os.path.join(train_dir, 'PVC')  
train_rbb_dir = os.path.join(train_dir, 'RBB')  


validation_apc_dir = os.path.join(validation_dir, 'APC')  
validation_lbb_dir = os.path.join(validation_dir, 'LBB')  
validation_nor_dir = os.path.join(validation_dir, 'NOR')    
validation_pvc_dir = os.path.join(validation_dir, 'PVC')  
validation_rbb_dir = os.path.join(validation_dir, 'RBB')   


num_apc_tr = len(os.listdir('/content/division/train/APC'))
num_lbb_tr = len(os.listdir('/content/division/train/LBB'))
num_nor_tr = len(os.listdir('/content/division/train/NOR'))
num_pvc_tr = len(os.listdir('/content/division/train/PVC'))
num_rbb_tr = len(os.listdir('/content/division/train/RBB'))


num_apc_val = len(os.listdir('/content/division/val/APC'))
num_lbb_val = len(os.listdir('/content/division/val/LBB'))
num_nor_val = len(os.listdir('/content/division/val/NOR'))
num_pvc_val = len(os.listdir('/content/division/val/PVC'))
num_rbb_val = len(os.listdir('/content/division/val/RBB'))


total_train = num_apc_tr + num_lbb_tr + num_nor_tr  + num_pvc_tr + num_rbb_tr  
total_val = num_apc_val + num_lbb_val + num_nor_val  + num_pvc_val + num_rbb_val 

print(total_train)

batch_size = 128
epochs = 30
IMG_HEIGHT = 256
IMG_WIDTH = 256



In [ ]:

train_image_generator = ImageDataGenerator(rescale=1./255,
                                          width_shift_range=0.3) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255)
                                               #width_shift_range=0.3) # Generator for our validation data
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           color_mode="grayscale",
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')
print(len(train_data_gen.classes[0:]))

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              color_mode="grayscale",
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')


test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=test_dir,
                                                           color_mode="grayscale",
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

In [ ]:
model = Sequential([
    Conv2D(8, (4,4),strides = (1,1), activation='relu', input_shape = (IMG_HEIGHT, IMG_WIDTH ,1),kernel_initializer='glorot_uniform'),
    #BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides= (2,2)),
    Conv2D(13, (2,2), activation='relu', strides = (1,1)),
    #BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides= (2,2)),
    Conv2D(13, (2,2), activation='relu', strides = (1,1)),
    #BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides= (2,2)),
    Flatten(),
    Dropout(0.4),
    Dense(1024, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(5, activation='softmax')
])


model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()


In [ ]:

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    #callbacks=[cp_callback]
)

In [ ]:
model.save('avnish_new_proj.h5')

In [ ]:
loss, acc = model.evaluate(test_data_gen)

In [ ]:
plt.plot(history.history['loss'], color='b')
plt.plot(history.history['val_loss'], color='r')
plt.show()
plt.plot(history.history['acc'], color='b')
plt.plot(history.history['val_acc'], color='r')
plt.show()

In [ ]:
import cv2
test_predict = []
test_original = []
for i in os.listdir('/content/drive/My Drive/Colab Notebooks/images_test/PVC'):
  loc = '/content/drive/My Drive/Colab Notebooks/images_test/PVC/' + i
  img = cv2.imread(loc)
  im = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  im = im.reshape((-1, 256, 256, 1))
  
  test_predict.append(np.argmax(model.predict(im)))


In [ ]:
y_hat = model.predict(test_data_gen)

In [ ]:
a = np.argmax(y_hat, axis = 1)

In [ ]:
a